In [1]:
import pandas as pd
from spark_session import LocalSparkSession
from dataset import Dataset
from mr_id3 import MapReduceIDR3
from pyspark.mllib.tree import DecisionTree

In [2]:
from log import log
from sklearn.impute import SimpleImputer
from pyspark.sql.functions import lit
from pyspark.sql.functions import col, sum
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [3]:
%time
num_fields = [
    'age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
    'hours_per_week', ]

categorical_fields = [
    'workclass', 'education',
    'marital_status', 'occupation', 'relationship',
    'race', 'sex', 'native_country', ]

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs


In [4]:
%time
target = 'label'
filename = 'dataset/adult.data'

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


In [5]:
%time
clusters = [1, 2, 3, 4, ]  # list(range(1, 2))
multiply = [5, ]  # list(range(1, 2))
metrics = []

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


In [6]:
# spark = LocalSparkSession(2)
# spark.start()

In [7]:
# dataset = Dataset(spark.spark, filename, num_fields, categorical_fields, target)
# dataset.load()
# dataset.string_indexer()

In [8]:
# 1/0

In [9]:
# dataset.df.columns

In [10]:
# 1/0

In [11]:
# df = dataset.df

In [12]:
# df.show(2)

In [13]:
# from pyspark.mllib.linalg import Vectors
# from pyspark.sql.types import Row

In [14]:
# df.rdd.map(lambda r: Row(
#         label = r.label,
# #         age = r.age,
# #         fnlwgt = r.fnlwgt,
# #         education_num = r.education_num,
# #         capital_gain = r.capital_gain,
# #         capital_loss = r.capital_loss,
# #         hours_per_week = r.hours_per_week,
# #         workclass_idx = r.workclass_idx,
# #         education_idx = r.education_idx,
# #         marital_status_idx = r.marital_status_idx,
# #         occupation_idx = r.occupation_idx,
# #         relationship_idx = r.relationship_idx,
# #         race_idx = r.race_idx,
# #         sex_idx = r.sex_idx,
# #         native_country_idx = r.native_country_idx,
#         features = Vectors.dense(
#             r.age, 
#             r.fnlwgt, 
#             r.education_num, 
#             r.capital_gain, 
#             r.capital_loss, 
#             r.hours_per_week, 
#             r.workclass_idx, 
#             r.education_idx, 
#             r.marital_status_idx, 
#             r.occupation_idx, 
#             r.relationship_idx, 
#             r.race_idx, 
#             r.sex_idx, 
#             r.native_country_idx
#         ))
#         ).toDF().show()

In [15]:
# df.rdd.map(lambda r: Row(
#                     label = r.label,
#                     age = r.age,
#                     features = Vectors.dense(r.label, r.age))
#                     ).toDF().show()

In [16]:
# df.withColumn("features", Vectors.dense([df.label, df.age])) \
#   .show()

In [17]:
# dataset.string_indexer()


In [18]:
# dataset.assemble_features()

In [19]:
# dataset.df.show(5)

In [20]:
# dataset.df.toPandas().head(5)

In [21]:
# 1/0

In [22]:
# mr_id3 = MapReduceIDR3(dataset)

In [23]:
# mr_id3.set_labeled_point()

In [24]:
# mr_id3.split()

In [25]:
# mr_id3.dataset.df.schema

In [26]:
# mr_id3.training_data.toDF()

In [27]:
# mr_id3.train()

In [28]:
# mr_id3.model = DecisionTree.trainClassifier(
#             mr_id3.training_data,
#             numClasses=2,
#             categoricalFeaturesInfo={},
#             impurity='entropy',
#             maxDepth=5,
#             maxBins=42,
#         )

In [29]:
# mr_id3.predictions = mr_id3.model.predict(
#             mr_id3.test_data.map(lambda x: x.features))

In [30]:
# labels_and_predictions = mr_id3.test_data.map(
#             lambda lp: lp.label).zip(mr_id3.predictions)

In [31]:
# dfl = labels_and_predictions.toDF()
# dfl.printSchema()
# dfl.show()

In [32]:
# 1/0 #labels_and_predictions

In [33]:
# mr_id3.errors = labels_and_predictions.filter(
#             lambda lp: lp[0] != lp[1]).count() / float(mr_id3.test_data.count())

In [34]:
# metrics = BinaryClassificationMetrics(labels_and_predictions)

In [35]:
# from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [36]:
# mr_id3.predict()

In [37]:
# mr_id3 = MapReduceIDR3(dataset)

In [38]:
# dataset.df.show(5)

In [39]:
# 1/0

In [40]:
%time
for n_clusters in clusters:
    for m_factor in multiply:
        spark = LocalSparkSession(n_clusters)
        spark.start()

        dataset = Dataset(spark.spark, filename, num_fields, categorical_fields, target)
        dataset.load()
        dataset.one_hot_encode_categorical_fields()
        dataset.multiply_dataset(m_factor)

        mr_id3 = MapReduceIDR3(dataset)
        mr_id3.set_labeled_point()
        mr_id3.split()
        mr_id3.train()
        mr_id3.predict()
        
        metric = mr_id3.get_metrics()
        metric['length_rows'] = dataset.df.count()
        metric['m_factor'] = m_factor
        metric['n_clusters'] = n_clusters
        metrics.append(metric)

        spark.stop()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/14 07:45:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/14 07:45:08 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


22/10/14 07:45:23 WARN DAGScheduler: Broadcasting large task binary with size 1147.2 KiB


22/10/14 07:45:28 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 24 (TID 16): Attempting to kill Python Worker


...

22/10/14 10:02:33 WARN DAGScheduler: Broadcasting large task binary with size 1157.6 KiB
22/10/14 10:02:34 WARN DAGScheduler: Broadcasting large task binary with size 1158.7 KiB
22/10/14 10:02:34 WARN DAGScheduler: Broadcasting large task binary with size 1160.7 KiB


22/10/14 10:02:35 WARN DAGScheduler: Broadcasting large task binary with size 1162.5 KiB


22/10/14 10:04:12 WARN DAGScheduler: Broadcasting large task binary with size 1148.4 KiB


22/10/14 10:04:56 WARN DAGScheduler: Broadcasting large task binary with size 1160.3 KiB


22/10/14 10:04:59 WARN DAGScheduler: Broadcasting large task binary with size 1172.3 KiB


In [41]:
%time
df = pd.DataFrame.from_dict(metrics)

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 12.2 µs


In [42]:
df['time', ']

,time,errors,area_under_pr,area_under_roc,length_rows,m_factor,n_clusters
0,141.487169,0.174388,0.410725,0.756192,1041952,5,1
1,83.355127,0.170787,0.421962,0.770086,1041952,5,2
2,72.910038,0.165940,0.437139,0.787724,1041952,5,3
3,2475.912175,0.166134,0.439570,0.793499,1041952,5,4
